In [ ]:
# docker run -v ${PWD}:/home/jovyan/work -p 8888:8888 --rm jupyter/pyspark-notebook

In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.getOrCreate()

In [3]:
df = spark.read.csv('../jovyan/work/API/mushrooms.csv',header=True,inferSchema=True)

In [4]:
df.show()
df.printSchema()

+-----+---------+-----------+---------+-------+----+---------------+------------+---------+----------+-----------+----------+------------------------+------------------------+----------------------+----------------------+---------+----------+-----------+---------+-----------------+----------+-------+
|class|cap-shape|cap-surface|cap-color|bruises|odor|gill-attachment|gill-spacing|gill-size|gill-color|stalk-shape|stalk-root|stalk-surface-above-ring|stalk-surface-below-ring|stalk-color-above-ring|stalk-color-below-ring|veil-type|veil-color|ring-number|ring-type|spore-print-color|population|habitat|
+-----+---------+-----------+---------+-------+----+---------------+------------+---------+----------+-----------+----------+------------------------+------------------------+----------------------+----------------------+---------+----------+-----------+---------+-----------------+----------+-------+
|    p|        x|          s|        n|      t|   p|              f|           c|        n|   

In [5]:
df.describe().show(vertical=True)

-RECORD 0--------------------------
 summary                  | count  
 class                    | 8124   
 cap-shape                | 8124   
 cap-surface              | 8124   
 cap-color                | 8124   
 bruises                  | 8124   
 odor                     | 8124   
 gill-attachment          | 8124   
 gill-spacing             | 8124   
 gill-size                | 8124   
 gill-color               | 8124   
 stalk-shape              | 8124   
 stalk-root               | 8124   
 stalk-surface-above-ring | 8124   
 stalk-surface-below-ring | 8124   
 stalk-color-above-ring   | 8124   
 stalk-color-below-ring   | 8124   
 veil-type                | 8124   
 veil-color               | 8124   
 ring-number              | 8124   
 ring-type                | 8124   
 spore-print-color        | 8124   
 population               | 8124   
 habitat                  | 8124   
-RECORD 1--------------------------
 summary                  | mean   
 class                    | 

In [6]:
from pyspark.sql.functions import when, count, col, isnull

df.select([count(when(isnull(c), c)).alias(c) for c in df.columns]).show()

+-----+---------+-----------+---------+-------+----+---------------+------------+---------+----------+-----------+----------+------------------------+------------------------+----------------------+----------------------+---------+----------+-----------+---------+-----------------+----------+-------+
|class|cap-shape|cap-surface|cap-color|bruises|odor|gill-attachment|gill-spacing|gill-size|gill-color|stalk-shape|stalk-root|stalk-surface-above-ring|stalk-surface-below-ring|stalk-color-above-ring|stalk-color-below-ring|veil-type|veil-color|ring-number|ring-type|spore-print-color|population|habitat|
+-----+---------+-----------+---------+-------+----+---------------+------------+---------+----------+-----------+----------+------------------------+------------------------+----------------------+----------------------+---------+----------+-----------+---------+-----------------+----------+-------+
|    0|        0|          0|        0|      0|   0|              0|           0|        0|   

In [7]:
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.feature import StringIndexer

In [8]:
features_list = []
indexer_list = []
for col in df.columns:
    indexer=StringIndexer(inputCol=col,outputCol=f"{col}_indexed")
    if col == 'class':
        indexer_fitted = indexer.fit(df)
        df_r=indexer_fitted.transform(df)
    else:
        indexer_fitted = indexer.fit(df_r)
        df_r=indexer_fitted.transform(df_r)
        features_list.append(f"{col}_indexed")
    indexer_list.append(indexer_fitted)

In [9]:
indexer_list

[StringIndexerModel: uid=StringIndexer_96abb39ef43b, handleInvalid=error,
 StringIndexerModel: uid=StringIndexer_ec0e3bd0f002, handleInvalid=error,
 StringIndexerModel: uid=StringIndexer_b87c34460369, handleInvalid=error,
 StringIndexerModel: uid=StringIndexer_6dc2b307329b, handleInvalid=error,
 StringIndexerModel: uid=StringIndexer_2d6fa7603d4a, handleInvalid=error,
 StringIndexerModel: uid=StringIndexer_0c4dde4ea289, handleInvalid=error,
 StringIndexerModel: uid=StringIndexer_e8c0961c6202, handleInvalid=error,
 StringIndexerModel: uid=StringIndexer_ece2e5d1d6de, handleInvalid=error,
 StringIndexerModel: uid=StringIndexer_38f68cf2edd6, handleInvalid=error,
 StringIndexerModel: uid=StringIndexer_2130d6f01392, handleInvalid=error,
 StringIndexerModel: uid=StringIndexer_97eeea233ec4, handleInvalid=error,
 StringIndexerModel: uid=StringIndexer_59b93203ec1d, handleInvalid=error,
 StringIndexerModel: uid=StringIndexer_6ae6eb58439c, handleInvalid=error,
 StringIndexerModel: uid=StringIndexer

In [10]:
df_r.head(5)

[Row(class='p', cap-shape='x', cap-surface='s', cap-color='n', bruises='t', odor='p', gill-attachment='f', gill-spacing='c', gill-size='n', gill-color='k', stalk-shape='e', stalk-root='e', stalk-surface-above-ring='s', stalk-surface-below-ring='s', stalk-color-above-ring='w', stalk-color-below-ring='w', veil-type='p', veil-color='w', ring-number='o', ring-type='p', spore-print-color='k', population='s', habitat='u', class_indexed=1.0, cap-shape_indexed=0.0, cap-surface_indexed=1.0, cap-color_indexed=0.0, bruises_indexed=1.0, odor_indexed=6.0, gill-attachment_indexed=0.0, gill-spacing_indexed=0.0, gill-size_indexed=1.0, gill-color_indexed=7.0, stalk-shape_indexed=1.0, stalk-root_indexed=2.0, stalk-surface-above-ring_indexed=0.0, stalk-surface-below-ring_indexed=0.0, stalk-color-above-ring_indexed=0.0, stalk-color-below-ring_indexed=0.0, veil-type_indexed=0.0, veil-color_indexed=0.0, ring-number_indexed=0.0, ring-type_indexed=0.0, spore-print-color_indexed=2.0, population_indexed=2.0, ha

In [11]:
from pyspark.ml.feature import VectorAssembler

In [12]:
features_list

['cap-shape_indexed',
 'cap-surface_indexed',
 'cap-color_indexed',
 'bruises_indexed',
 'odor_indexed',
 'gill-attachment_indexed',
 'gill-spacing_indexed',
 'gill-size_indexed',
 'gill-color_indexed',
 'stalk-shape_indexed',
 'stalk-root_indexed',
 'stalk-surface-above-ring_indexed',
 'stalk-surface-below-ring_indexed',
 'stalk-color-above-ring_indexed',
 'stalk-color-below-ring_indexed',
 'veil-type_indexed',
 'veil-color_indexed',
 'ring-number_indexed',
 'ring-type_indexed',
 'spore-print-color_indexed',
 'population_indexed',
 'habitat_indexed']

In [13]:
featureassembler=VectorAssembler(inputCols=features_list,outputCol="X")
output=featureassembler.transform(df_r)

In [14]:
print(len(output.columns))
output.select('X').show()

47
+--------------------+
|                   X|
+--------------------+
|(22,[1,3,4,7,8,9,...|
|(22,[1,2,3,4,8,9,...|
|(22,[0,1,2,3,4,8,...|
|(22,[2,3,4,7,8,9,...|
|(22,[1,2,6,8,10,1...|
|(22,[2,3,4,8,9,10...|
|(22,[0,1,2,3,4,8,...|
|(22,[0,2,3,4,8,9,...|
|(22,[2,3,4,7,8,9,...|
|(22,[0,1,2,3,4,8,...|
|(22,[2,3,4,8,9,10...|
|(22,[2,3,4,8,9,10...|
|(22,[0,1,2,3,4,8,...|
|(22,[2,3,4,7,8,9,...|
|(22,[1,6,8,10,12,...|
|(22,[0,1,2,7,8,9,...|
|(22,[0,1,2,6,8,10...|
|(22,[1,3,4,7,8,9,...|
|(22,[2,3,4,7,8,9,...|
|(22,[1,3,4,7,8,9,...|
+--------------------+
only showing top 20 rows



In [15]:
train_data,test_data=output.randomSplit([0.75,0.25])

In [16]:
classifier = LogisticRegression(featuresCol='X', labelCol='class_indexed')
classifier = classifier.fit(train_data)

In [17]:
results = classifier.evaluate(test_data)

In [18]:
results.accuracy

0.990234375

In [19]:
from pyspark.ml.classification import RandomForestClassifier

In [20]:
classifier_2 = RandomForestClassifier(featuresCol='X', labelCol='class_indexed')
model =  classifier_2.fit(train_data)

In [21]:
predictions = model.transform(test_data)
predictions.select('class_indexed', 'rawPrediction', 'probability', 'prediction').show()

+-------------+--------------------+--------------------+----------+
|class_indexed|       rawPrediction|         probability|prediction|
+-------------+--------------------+--------------------+----------+
|          0.0|[19.7129943415285...|[0.98564971707642...|       0.0|
|          0.0|[19.5264805550147...|[0.97632402775073...|       0.0|
|          0.0|[19.7153866860261...|[0.98576933430130...|       0.0|
|          0.0|[19.6207920914315...|[0.98103960457157...|       0.0|
|          0.0|[19.7160494889393...|[0.98580247444696...|       0.0|
|          0.0|[19.7160494889393...|[0.98580247444696...|       0.0|
|          0.0|[19.7129943415285...|[0.98564971707642...|       0.0|
|          0.0|[19.0152251437593...|[0.95076125718796...|       0.0|
|          0.0|[19.7153866860261...|[0.98576933430130...|       0.0|
|          0.0|[19.0176174882569...|[0.95088087441284...|       0.0|
|          0.0|[19.7153866860261...|[0.98576933430130...|       0.0|
|          0.0|[19.6207920914315..

In [22]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

In [23]:
evaluator = BinaryClassificationEvaluator(labelCol="class_indexed", rawPredictionCol="prediction")
result = evaluator.evaluate(predictions, {evaluator.metricName: "areaUnderROC"})

In [24]:
result

0.9969635627530364

In [28]:
model.save(path="../jovyan/work/API/model")

In [21]:
from pyspark.ml.classification import RandomForestClassificationModel

In [22]:
model = RandomForestClassificationModel.load("../jovyan/work/API/model")

In [23]:
predictions = model.transform(test_data)
predictions.select('class_indexed', 'rawPrediction', 'probability', 'prediction').show()

+-------------+--------------------+--------------------+----------+
|class_indexed|       rawPrediction|         probability|prediction|
+-------------+--------------------+--------------------+----------+
|          0.0|[19.7129943415285...|[0.98564971707642...|       0.0|
|          0.0|[19.6207920914315...|[0.98103960457157...|       0.0|
|          0.0|[19.8106440835339...|[0.99053220417669...|       0.0|
|          0.0|[19.7129943415285...|[0.98564971707642...|       0.0|
|          0.0|[19.0152251437593...|[0.95076125718796...|       0.0|
|          0.0|[19.6183997469339...|[0.98091998734669...|       0.0|
|          0.0|[19.7153866860261...|[0.98576933430130...|       0.0|
|          0.0|[19.7153866860261...|[0.98576933430130...|       0.0|
|          0.0|[19.6207920914315...|[0.98103960457157...|       0.0|
|          0.0|[19.8106440835339...|[0.99053220417669...|       0.0|
|          0.0|[19.8106440835339...|[0.99053220417669...|       0.0|
|          0.0|[19.7160494889393..

In [112]:
from pyspark.ml.classification import GBTClassifier

In [113]:
classifier_3 = GBTClassifier(featuresCol='X', labelCol='class_indexed')
model =  classifier_3.fit(train_data)

In [114]:
predictions = model.transform(test_data)
predictions.select('class_indexed', 'rawPrediction', 'probability', 'prediction').show()

+-------------+--------------------+--------------------+----------+
|class_indexed|       rawPrediction|         probability|prediction|
+-------------+--------------------+--------------------+----------+
|          0.0|[1.54267331840267...|[0.95628424516656...|       0.0|
|          0.0|[1.54267331840267...|[0.95628424516656...|       0.0|
|          0.0|[1.54267331840267...|[0.95628424516656...|       0.0|
|          0.0|[1.54267331840267...|[0.95628424516656...|       0.0|
|          0.0|[1.54267331840267...|[0.95628424516656...|       0.0|
|          0.0|[1.54267331840267...|[0.95628424516656...|       0.0|
|          0.0|[1.54267331840267...|[0.95628424516656...|       0.0|
|          0.0|[1.54267331840267...|[0.95628424516656...|       0.0|
|          0.0|[1.54267331840267...|[0.95628424516656...|       0.0|
|          0.0|[1.54267331840267...|[0.95628424516656...|       0.0|
|          0.0|[1.54267331840267...|[0.95628424516656...|       0.0|
|          0.0|[1.54267331840267..

In [124]:
model.featureImportances

SparseVector(22, {0: 0.0032, 1: 0.0006, 2: 0.0, 4: 0.9374, 5: 0.0, 6: 0.0, 7: 0.0009, 8: 0.0, 9: 0.0, 10: 0.0036, 11: 0.0018, 13: 0.0, 14: 0.0122, 18: 0.0, 19: 0.032, 20: 0.0079, 21: 0.0003})

In [115]:
evaluator = BinaryClassificationEvaluator(labelCol="class_indexed", rawPredictionCol="prediction")
result = evaluator.evaluate(predictions, {evaluator.metricName: "areaUnderROC"})

In [116]:
result

1.0

In [125]:
from pyspark.ml.classification import MultilayerPerceptronClassifier

In [160]:
classifier_4 = MultilayerPerceptronClassifier(layers=[22, 16, 8, 4, 2], featuresCol='X', labelCol='class_indexed')
model = classifier_4.fit(train_data)

predictions = model.transform(test_data)

In [161]:
predictions

DataFrame[class: string, cap-shape: string, cap-surface: string, cap-color: string, bruises: string, odor: string, gill-attachment: string, gill-spacing: string, gill-size: string, gill-color: string, stalk-shape: string, stalk-root: string, stalk-surface-above-ring: string, stalk-surface-below-ring: string, stalk-color-above-ring: string, stalk-color-below-ring: string, veil-type: string, veil-color: string, ring-number: string, ring-type: string, spore-print-color: string, population: string, habitat: string, class_indexed: double, cap-shape_indexed: double, cap-surface_indexed: double, cap-color_indexed: double, bruises_indexed: double, odor_indexed: double, gill-attachment_indexed: double, gill-spacing_indexed: double, gill-size_indexed: double, gill-color_indexed: double, stalk-shape_indexed: double, stalk-root_indexed: double, stalk-surface-above-ring_indexed: double, stalk-surface-below-ring_indexed: double, stalk-color-above-ring_indexed: double, stalk-color-below-ring_indexed:

In [162]:
predictions.select('prediction')

DataFrame[prediction: double]

In [163]:
evaluator = BinaryClassificationEvaluator(labelCol="class_indexed", rawPredictionCol="prediction")
result = evaluator.evaluate(predictions, {evaluator.metricName: "areaUnderROC"})

In [164]:
result

1.0